In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!git clone https://github.com/NidhiAI/Librosa_wav_to_features

Cloning into 'Librosa_wav_to_features'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 59 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), 2.37 MiB | 9.58 MiB/s, done.


In [2]:
import librosa
import pandas as pd
import numpy as np
import os
import csv
#from PIL import Image
#import pathlib
#import csv
#from sklearn.model_selection import train_test_split


#from sklearn.preprocessing import LabelEncoder, StandardScaler
#import tensorflow as tf
#from tensorflow import keras

#from keras import layers



Creating a header for our CSV file.

In [3]:
# create headers 
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [4]:

csvFilePath = "/kaggle/working/Librosa_wav_to_features/generatedFeatures/"
file = open(csvFilePath+'librosa.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
#genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
genres = 'male female'.split()
for g in genres:
    for filename in os.listdir(f'/kaggle/working/Librosa_wav_to_features/librosa_mp3/{g}'):
        songname = f'/kaggle/working/Librosa_wav_to_features/librosa_mp3/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        #rmse = librosa.feature.rmse(y=y)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open(csvFilePath+'librosa.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

Data preprocessing: It involves loading CSV data, label encoding, feature scaling and data split into training and test set.

In [5]:
data = pd.read_csv('/kaggle/working/Librosa_wav_to_features/generatedFeatures/librosa.csv')
data.head() 

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,Amir_Khan-1.wav,0.346160,0.086333,982.631579,1220.290604,1926.350394,0.039235,-262.391724,181.574829,-13.865875,...,-0.095696,3.396923,-12.072077,8.895560,-6.202807,-4.991406,-4.346209,-1.143554,-4.781991,male
1,Amitabh_Bachchan-1.wav,0.400863,0.014618,803.082429,1065.387798,1374.635824,0.036154,-458.852997,166.025391,31.336935,...,0.506921,6.600809,-0.597324,5.325176,2.121905,-1.066245,2.724939,0.938227,0.421700,male
2,Akshay_Kumar-1.wav,0.326013,0.061412,1242.978264,1383.492947,2211.111630,0.058998,-271.693207,144.071121,-20.575047,...,-8.720441,2.798906,-8.777470,-4.245789,-2.628192,-4.956270,-6.886003,-4.317806,-4.395898,male
3,Om_Puri-1.wav,0.451828,0.015613,1448.876384,1635.622133,3064.825622,0.071417,-387.628296,145.813339,-15.173584,...,-6.585880,6.340422,-10.105690,8.014238,-6.661406,5.122551,0.846031,0.162425,4.523784,male
4,John_Abraham-1.wav,0.318019,0.087968,1218.571704,1118.675713,2237.372185,0.080105,-262.468018,176.511597,-44.926266,...,-19.286688,-5.887372,-13.047683,-9.813202,6.346204,-18.739126,-1.797028,-4.955317,-12.663363,male


In [7]:
data.shape # all 14 audio files converted to features 

(14, 28)

In [8]:
data.label.unique()

array(['male', 'female'], dtype=object)

In [9]:
data.label.value_counts()

male      8
female    6
Name: label, dtype: int64